In [4]:
data = "389125467"

In [5]:
def repr_cups(succ, start):
    vals = [start]
    i = succ[start]
    while i != start:
        vals.append(i)
        i = succ[i]
    return "".join([str(i+1) for i in vals])

In [6]:
# def crab_cups(data, n_iter):
#     cups = [int(n)-1 for n in data]
#     N = len(cups)
#     current = cups[0]
#     succ = cups[1:]+cups[:1]
#     pred = cups[-1:]+cups[:-1]
#     succ = {c: s for c, s in zip(cups, succ)}
#     succ = {c: p for c, p in zip(cups, prev)}
    
#     for _ in range(n_iter):
#         moved = []
#         p = current
#         for _ in range(3):
#             moved.append(succ[p])
#             p = moved[-1]
#         dest = (current - 1) % N
#         while dest in moved:
#             dest = (dest - 1) % N
#         print("> ", current, moved, dest)
#         succ[current], succ[dest], succ[moved[-1]] = succ[moved[-1]], moved[0], succ[dest]
        
#         succ[current] = succ[moved[-1]]
#         succ[dest] = moved[0]
#         succ[moved[-1]] = succ[dest]
#         pred[moved[0]] = dest
#         succ[pred[dest]] = 
        
        
#         current = succ[current]
#         print(current+1)
#         print_cups(succ, current)
        

In [7]:
def crab_cups(data, n_iter):
    cups = [int(n)-1 for n in data]
    N = len(cups)
    current = cups[0]
    succ = np.zeros(N, dtype=np.uint8)
    for c, s in zip(cups, cups[1:]+cups[:1]):
        succ[c] = s
    
    for _ in range(n_iter):
        moved = []
        p = current
        for _ in range(3):
            moved.append(succ[p])
            p = moved[-1]
        dest = (current - 1) % N
        while dest in moved:
            dest = (dest - 1) % N
        succ[current], succ[dest], succ[moved[-1]] = succ[moved[-1]], moved[0], succ[dest]
        current = succ[current]
    return int(repr_cups(succ, 0)[1:])
        

In [8]:
crab_cups(data, 100)

67384529

In [6]:
crab_cups("362981754", 100)

24798635

In [186]:
from numba import njit, jit

In [187]:
def parse(data):
    cups = np.array([int(n)-1 for n in data])
    return cups

In [193]:
@njit
def crab_cups_hard(cups, n_iter, N):
    current = cups[0]
    succ = np.arange(1,N+1, dtype=np.uint32)
    for c, s in zip(cups, np.roll(cups,-1)):
        succ[c] = s
    if len(cups) < N:
        succ[-1], succ[cups[-1]] = cups[0], len(cups)

    for _ in range(n_iter):
        moved = []
        p = current
        for _ in range(3):
            moved.append(succ[p])
            p = moved[-1]
        dest = (current - 1) % N
        while dest in moved:
            dest = (dest - 1) % N
        succ[current], succ[dest], succ[moved[-1]] = succ[moved[-1]], moved[0], succ[dest]
        current = succ[current]
    return (succ[0]+1)*(succ[succ[0]]+1)

In [194]:
import time

In [195]:
%%timeit
t0 = time.time()
crab_cups_hard(parse("389125467"), int(1e7), int(1e6))
print(time.time()-t0)

5.863401889801025
5.85166597366333
5.733330965042114
5.925648927688599
6.277633190155029
6.69622278213501
7.493098974227905
7.261867046356201
6.46 s ± 653 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [191]:
crab_cups_hard(parse("362981754"), int(1e7), int(1e6))

12757828710

In [164]:
from time import time

my_input = "215694783"
example = "389125467"

def play(cups, iterations):
    max_ = max(cups)
    circle = [0] * (max_+1) # thank you /u/setapoux/
    for i in range(len(cups)-1):
        circle[cups[i]] = cups[i+1]
    circle[cups[-1]] = cups[0]
    current = circle[cups[-1]]

    for _ in range(iterations):
        picked_up = []
        next_ = current
        for _ in range(3):
            next_ = circle[next_]
            picked_up.append(next_)

        destination = current
        while True:
            destination -= 1
            if destination < 1:
                destination = int(max_)
            if destination not in picked_up:
                break

        temp = circle[picked_up[-1]]
        circle[picked_up[-1]] = circle[destination]
        circle[destination] = picked_up[0]
        circle[current] = temp

        current = temp # thank you /u/setapoux/

    a = circle[1]
    b = circle[a]
    print(a*b)

import time
t0 = time.time()
play(list(map(int, list(my_input))) + list(range(10, int(1e6)+1)), int(1e7))
print(time.time()-t0)

163035127721
12.864352941513062
